In [1]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings 
import random
from sklearn.preprocessing import scale
warnings.filterwarnings("ignore", category=DeprecationWarning)

%matplotlib inline

df = pd.read_csv('training_cleaned.csv')

df["Sentiment"] = df["Sentiment"].map({4:1,0:0})

df.head()
SEED = random.seed(100)
random.seed(100)

In [2]:
df_new = df.loc[df['Tweet'].apply(type) == str].copy()

In [3]:
x = df_new.Tweet
y = df_new.Sentiment

In [4]:
#Create Train,Validate and TEST Sets (80,10,10)
from sklearn.model_selection import train_test_split

x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.2, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test,
                                                              y_validation_and_test, test_size=.5, random_state=SEED)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,classification_report
from sklearn.metrics import classification_report, confusion_matrix
from time import time

In [ ]:
import gensim.downloader as api

#function to create one vector for multiple words by taking average vector of words or sum of vectors

def get_w2v_general(tweet, size, vectors, aggregation='mean'):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tweet.split():
        try:
            vec += vectors[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if aggregation == 'mean':
        if count != 0:
            vec /= count
        return vec
    elif aggregation == 'sum':
        return vec
    

c:\users\tblakeley\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
#Load pre built GLOVE model
glove_twitter = api.load("glove-twitter-200")

In [ ]:
#create single tweet vector by averaging each word vector
vecs_glove_mean = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'mean') for z in df_new["Tweet"]]))
#create single tweet vector by summing each word vector
vecs_glove_sum = scale(np.concatenate([get_w2v_general(z, 200, glove_twitter,'sum') for z in df_new["Tweet"]]))

In [ ]:
glove_x_train, glove_x_validation_and_test, glove_y_train, glove_y_validation_and_test = train_test_split(vecs_glove_mean,
                                                                                                          y, test_size=.2, random_state=SEED)
glove_x_validation, glove_x_test, glove_y_validation, glove_y_test = train_test_split(glove_x_validation_and_test,
                                                                                      glove_y_validation_and_test, test_size=.5, random_state=SEED)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_predict
names = ["Logistic Regression", "Linear SVC"#,"Multinomial NB", 
          ,"RandomForest", "AdaBoost","KNN","XGBOOST","STACKClassifier"]
classifiers = [
    LogisticRegression(solver='liblinear'),
    LinearSVC(),
    RandomForestClassifier(n_estimators=200),
    AdaBoostClassifier(),
    KNeighborsClassifier(),
    XGBClassifier(max_depth=30),
    StackingClassifier(classifiers=[LogisticRegression(),RandomForestClassifier(n_estimators=200)
                                    ,XGBClassifier(max_depth=30)], 
                          meta_classifier=LogisticRegression(solver='liblinear'))
    ]
zipped_clf = zip(names,classifiers)


In [ ]:

df_All = pd.DataFrame()



for n,c in zipped_clf:
    t0 = time()
    

    # Fit and tune model
    #print(clf.estimator)

    
    clf = c.fit(glove_x_train,glove_y_train)
    train_test_time = time() - t0
    preds = clf.predict(glove_x_test)
    accuracy = accuracy_score(glove_y_test, preds)
    model_precision, model_recall, model_fscore, model_support = precision_recall_fscore_support(y_test,preds)
    #bestngram = clf.best_params_['vectorizer__ngram_range']
    #bestnfeatures = clf.best_params_['vectorizer__max_features']
    df_loop = pd.DataFrame([n,
                            #bestnfeatures,bestngram,
                            accuracy,model_precision[0],model_recall[0],model_fscore[0]])
    df_All = df_All.append(df_loop.T, ignore_index = True)
    print("Model: {0}".format(n))
    #print("Features: {0}".format(clf.best_params_))
    print("Accuracy: {0:.2f}%".format(accuracy*100))
    print("train and test time: {0:.2f}s".format(train_test_time))
    print("-"*80)
    df_All

In [ ]:
param_test1 = {
 'max_depth':range(3,10,2),
 'min_child_weight':range(1,6,2)not sure 
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5,verbose = 5)
gsearch1.fit(glove_x_train,glove_y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_